# Dataset.map()

This notebook shows a workflow for using `Dataset.map`.

`Dataset.map()` works like a standard map function, creating a new column using a python function that reads the existing rows, with extra API to allow the UI to visualize changes.


In [1]:
%load_ext autoreload
%autoreload 2
import lilac as ll

ll.set_project_dir('./data')

try:
  glue = ll.get_dataset('local', 'glue_ax_map')
except Exception as e:
  glue = ll.create_dataset(
    ll.DatasetConfig(
      namespace='local',
      name='glue_ax_map',
      source=ll.HuggingFaceSource(dataset_name='glue', config_name='ax', sample_size=100),
    )
  )

# ll.start_server()


/Users/nikhil/Code/lilac/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Simple `map`: upper case 'premise'

The following map will upper case the `premise` field from the dataset. When you have the [Lilac UI](http://localhost:5432) opened in a new tab, you will see the progress as your map is executed.

Once it's complete, refresh the UI to see the results.

The output of the map is also returned as a generator so you can print it easily. The results of the generator are simply what your map outputed.

You can use `select_rows` to merge it with source data.


In [2]:
# First, let's just run our map function that upper-cases the 'premise' field without writing a
# column to disk until we're happy with the results.
def _upper(row: dict, job_id: int) -> str:
  return row['premise'].upper()


res = glue.map(_upper)
print(next(iter(res)))
print()

# Once we're comfortable with the results, we can write the column to disk.
# To do this, we specify `output_column`.
glue.map(_upper, output_column='premise_upper', overwrite=True)

# Print the first three rows.
rows = glue.select_rows(['premise', 'premise_upper'], limit=3)
for row in rows:
  print(row)


Scheduling task "6c5b363446bf44b9b84b0321e5d7087f": "[local/glue_ax_map][shard 0/1] map "_upper" to "_upper"".


[local/glue_ax_map][shard 0/1] map "_upper" to "_upper": 100%|██████████| 1104/1104 [00:00<00:00, 55017.72it/s]


A RABBI IS AT THIS WEDDING, STANDING RIGHT THERE STANDING BEHIND THAT TREE.

Scheduling task "dee8d22356a2400aba4210667d937bf7": "[local/glue_ax_map][shard 0/1] map "_upper" to "premise_upper"".
Task finished "6c5b363446bf44b9b84b0321e5d7087f": "[local/glue_ax_map][shard 0/1] map "_upper" to "_upper"" in 5s.
Wrote map output to ./data/datasets/local/glue_ax_map/./data/datasets/local/glue_ax_map/premise_upper-00000-of-00001.parquet
{'premise': 'The cat sat on the mat.', 'premise_upper': 'THE CAT SAT ON THE MAT.'}
{'premise': "When you've got no snow, it's really hard to learn a snow sport so we looked at all the different ways I could mimic being on snow without actually being on snow.", 'premise_upper': "WHEN YOU'VE GOT NO SNOW, IT'S REALLY HARD TO LEARN A SNOW SPORT SO WE LOOKED AT ALL THE DIFFERENT WAYS I COULD MIMIC BEING ON SNOW WITHOUT ACTUALLY BEING ON SNOW."}
{'premise': "When you've got snow, it's really hard to learn a snow sport so we looked at all the different ways I could 

[local/glue_ax_map][shard 0/1] map "_upper" to "premise_upper": 100%|██████████| 1104/1104 [00:00<00:00, 51739.29it/s]


# Find all instances of a keyword and highlight it in the UI

We can also use the `nest_under` to nest the result under a specific field.

Once a field is nested, we can use `ll.span` to emit the character coordinates, with metadata, of a field we want to highlight from the UI.

In this example, we'll simply highlight keywords with "the". We'll emit metadata with the span so we can filter by it from the UI!

Run the next cell, and then open [this link](http://localhost:5432/datasets#local/glue_ax_map&schemaCollapsed=false&showMetadataPanel=true&expandedStats=%7B%22premise.thes%22%3Atrue%7D&query=%7B%22filters%22%3A%5B%7B%22path%22%3A%5B%22premise%22%2C%22thes%22%5D%2C%22op%22%3A%22equals%22%2C%22value%22%3A%22the%22%7D%5D%7D). You'll notice we had to apply the filter for 'the' before we get highlighting.


In [3]:
import re

# Find instances of 'the'.
MATCH_REGEX = 'the'


def _find_the(item: ll.Item) -> ll.Item:
  return [ll.span(m.start(), m.end()) for m in re.finditer(MATCH_REGEX, item['premise'])]


glue.map(
  _find_the, output_column='thes', nest_under='premise', overwrite=True, combine_columns=False
)

# Print the first three rows.
rows = glue.select_rows([ll.PATH_WILDCARD], combine_columns=False, limit=3)
for row in rows:
  print(row)


Scheduling task "207a165662a04040a58999335bd11bac": "[local/glue_ax_map][shard 0/1] map "_find_the" to "thes"".
Task finished "dee8d22356a2400aba4210667d937bf7": "[local/glue_ax_map][shard 0/1] map "_upper" to "premise_upper"" in 4s.
Wrote map output to ./data/datasets/local/glue_ax_map/./data/datasets/local/glue_ax_map/premise/thes/thes-00000-of-00001.parquet
{'premise': 'The cat sat on the mat.', 'hypothesis': 'The cat did not sit on the mat.', 'label': 'contradiction', 'idx': 0, '__hfsplit__': 'test', 'premise_upper': 'THE CAT SAT ON THE MAT.', 'premise.thes.*': [{'__span__': {'start': 15, 'end': 18}}]}
{'premise': "When you've got no snow, it's really hard to learn a snow sport so we looked at all the different ways I could mimic being on snow without actually being on snow.", 'hypothesis': "When you've got snow, it's really hard to learn a snow sport so we looked at all the different ways I could mimic being on snow without actually being on snow.", 'label': 'contradiction', 'idx'

[local/glue_ax_map][shard 0/1] map "_find_the" to "thes": 100%|██████████| 1104/1104 [00:00<00:00, 38167.75it/s]


Task finished "207a165662a04040a58999335bd11bac": "[local/glue_ax_map][shard 0/1] map "_find_the" to "thes"" in 4s.


# Map continuation during an error, or computer shutdown

`dataset.map()` will not lose data if an error is thrown when writing to disk.

The next time it is called, it will continue from where it left off. Once it is finally complete, the column is written.


In [9]:
throw_for_rowid = True

random_row_id = list(glue.select_rows([ll.ROWID], limit=1))[0][ll.ROWID]


def _upper(item: ll.Item):
  global i, throw_after_n
  if throw_for_rowid and item[ll.ROWID] == random_row_id:
    raise ValueError(f'Throwing for {random_row_id}')
  return item['premise'].upper()


# This is going to throw after 10 iterations. When we call it again, it will only call _upper()
# for the rest of the dataset.
glue.map(_upper, output_column='premise_upper2', overwrite=True, num_jobs=1)


Scheduling task "8720732957904b099176a9f0c7320de1": "[local/glue_ax_map][shard 0/1] map "_upper" to "premise_upper2"".


[local/glue_ax_map][shard 0/1] map "_upper" to "premise_upper2":  38%|███▊      | 414/1104 [00:00<00:00, 23549.44it/s]
2023-11-20 19:13:53,854 - distributed.worker - WARNING - Compute Failed
Key:       8720732957904b099176a9f0c7320de1
Function:  _execute_task
args:      (<function _upper at 0x2a7f7b920>, ('premise_upper2',), './data/.cache/lilac/local/glue_ax_map/premise_upper2.00000-of-00001.jsonl', 0, 1, True, False, False, ('8720732957904b099176a9f0c7320de1', 0))
kwargs:    {}
Exception: "ValueError('Throwing for 5c56d63b5f0a401696a8a20ac1f027fa')"



ValueError: Throwing for 5c56d63b5f0a401696a8a20ac1f027fa

Task error "8720732957904b099176a9f0c7320de1": "[local/glue_ax_map][shard 0/1] map "_upper" to "premise_upper2"" in 0s.


In [10]:
throw_for_rowid = False
# This will finish calling _upper, without calling it for the first 10 items.
glue.map(_upper, output_column='premise_upper2', num_jobs=1)


Scheduling task "4130416bb1a5464abebb69d7c6086920": "[local/glue_ax_map][shard 0/1] map "_upper" to "premise_upper2"".
Wrote map output to ./data/datasets/local/glue_ax_map/./data/datasets/local/glue_ax_map/premise_upper2-00000-of-00001.parquet


[local/glue_ax_map][shard 0/1] map "_upper" to "premise_upper2": 100%|██████████| 1104/1104 [00:00<00:00, 36751.49it/s]


Task finished "4130416bb1a5464abebb69d7c6086920": "[local/glue_ax_map][shard 0/1] map "_upper" to "premise_upper2"" in 0s.
